<a href="https://colab.research.google.com/github/KonstantinSV/1NN-VS-RandomForest/blob/main/Decision_1NN_VS_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import model_selection, datasets, metrics, ensemble
import numpy as np

In [ ]:
digits = datasets.load_digits()
print(f"digits: {digits.keys()}")

digits: dict_keys(['data', 'target', 'target_names', 'images', 'DESCR'])


In [ ]:
X = digits.data
y = digits.target

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, shuffle=False, test_size=0.25) # при выборе параметров test_size=0.25 или train_size=0.75 получабтся разные разбиения

Метод 1. Построение 1NN

In [ ]:
# фун-ия считает эвклидову метрику, но без извлечения корня. Корень влияет только на монотонность, но не на свойства признаков
def dist(v1, v2):
    return sum((v1 - v2)**2)

In [ ]:
# фун-ия возвращает кол-во предсказаний на тестовой выборке Х_test несовпавших с истинными метками y_test 
def targets(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test): # при последующем вызове ф-ию можно вызывать без параметров или с с меньшим числом параметров (меньше 4) так как они уже заданы X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test
    target_X_test = [] # список предсказаний на тестовой выборке Х_test
    for x_te in X_test: # x_te принимает значение первого объекта из тестовой выборки X_test (вектор со всеми признаками)
        arr=[] # Для каждого объекта из X_test список arr обнуляется
        """В ф-ию dist подается объект из тестовой выборки x_te и объект из обучающей выборки x_tr.
        Ф-ия dist вычитает один вектор из другого, возводит в кадрат и складывает, так определяется расстояние между объектами. 
        Чем меньше полученное скалярное значение, тем ближе объекты друг к другу.
        Так же к полученному скалярному произведению добавляется метка класса из обучающей выборки y_tr.
        На выходе из цикла поручаем список списков arr.
        Первый элемент в списках - скалярное расстоянияние от одного объекта из X_test до всех объектов X_train.
        Второй элемент - метка класса из из обучающей выборки y_train """
        arr = [[dist(x_te, x_tr), y_tr] for x_tr, y_tr in zip(X_train, y_train)] 
        arr.sort() # Сортируем по первому элементу списка списков. Ведь чем меньше расстояние между объектами, тем больше вероятность принадлежности его к указанному классу (то есть ко второй метке)
        target_X_test.append(arr[0][1]) # Добавляем в target_X_test только второй элемент порвого объекта, то усть метку класса y_train  с самым меньшим расстоянием
    return sum(y_test != target_X_test) # сравниваем список y_test со списком target_X_test (в котором метки классов y_train) и считаем сумму не совпавших элементов (17 элементов)

In [ ]:
# Ф-ия считает долю неправильных ответов.
# targets фозвращает кол-во не совпавших предсказаний (17 элементов) которое мы делим на кол-во объектов в тестовой выборке
targets()/X_test.shape[0]

0.03777777777777778

Метод 2. Построение 1NN

In [ ]:
# фун-ия считает эвклидову метрику. Возвращает скалярное значение? расстояние до точки из обучающей выборки/
def euclidian_metric(x, y):
    return np.sqrt( np.sum((x - y)**2) )

In [ ]:
y_pred_knn = [] # список предсказаний на тестовой выборке Х_test 
for test_value in X_test:
    ind_min_metric = 0 # ind_min_metric обнуляется при каждом цикле
    min_metric = euclidian_metric(test_value, X_train[0]) # вычисляется euclidian_metric только для первого объекта X_train[0], что бы далее сравнивать метриками остальных объектов X_train
    
    for index, train_value in enumerate(X_train): # enumerate возвращает порядковые номера объектов (index) и сами объекты (train_value)
        """В ф-ию euclidian_metric подается объект из тестовой выборки test_value и объект из обучающей выборки train_value.
        Здесь еще раз вычисляется метрика для X_train[0]
        Ф-ия euclidian_metric определяется расстояние между объектами. """
        metric = euclidian_metric(test_value, train_value)
        if metric < min_metric: # Определяется самая маленькая метрика между одним объектом из X_test и всеми объектами из X_train
            min_metric = metric 
            ind_min_metric = index # в переменную ind_min_metric сохраняется порядковый номер минимальной метрики
            
    y_pred_knn.append(y_train[ind_min_metric]) # добавляем в y_pred_knn метку класса из обучающей выборки y_train с индексом (порядковым номером) ind_min_metric

In [ ]:
# Ф-ия accuracy_score считает долю Правильных ответов. Чтобы посчитать долю Неправильных нужно от 1 отнять accuracy_score
knn_err_rate = 1 - metrics.accuracy_score(y_test, y_pred_knn)
knn_err_rate

0.0377777777777778

Построение RandomForest

In [ ]:
Random_Forest = ensemble.RandomForestClassifier(n_estimators=1000) # cоздаем лес c 1000 деревьями

In [ ]:
Random_Forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# Ф-ия accuracy_score считает долю Правильных ответов. Чтобы посчитать долю Неправильных нужно от 1 отнять accuracy_score
1 - metrics.accuracy_score(y_test, Random_Forest.predict(X_test)) # predict - предсказания леса для тестовой выборки.

0.06444444444444442

Результат:
Лес работат хуже самого простого kNN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы